In [1]:
import dotenv
dotenv.load_dotenv('../.env')
import db_operations

In [8]:
import os
import sqlalchemy as db

In [9]:
import sqlalchemy

In [12]:
def connect_unix_socket() -> sqlalchemy.engine.base.Engine:
    db_user = os.environ["DB_USER"]  # e.g. 'my-database-user'
    db_pass = os.environ["DB_PASS"]  # e.g. 'my-database-password'
    db_name = os.environ["DB_NAME"]  # e.g. 'my-database'
    unix_socket_path = os.environ["INSTANCE_UNIX_SOCKET"]  # e.g. '/cloudsql/project:region:instance'

    pool = sqlalchemy.create_engine(
        sqlalchemy.engine.url.URL.create(
            drivername="mysql+pymysql",
            username=db_user,
            password=db_pass,
            database=db_name,
            query={"unix_socket": unix_socket_path},
            port=3306
        )
    )
    return pool


In [13]:
engine = connect_unix_socket()

In [14]:
def get_table(name):
    return db.Table(name, db.MetaData(), autoload=True, autoload_with=engine)
def get_images_from_room(room):

    images = get_table('images')
    users = get_table('users')
    stmt = db.select(users.c.pseudo,
                     images.c.prompt,
                     images.c.height,
                     images.c.width,
                     images.c.posX,
                     images.c.posY,
                     images.c.timestamp,
                     images.c.status,
                     images.c.path).where(db.or_(users.c.email == images.c.email, images.c.email is None ,images.c.email is None)).where(images.c.room == room).where(images.c.status == 'generated')
    result = engine.execute(stmt)
    result_as_dict = result.mappings().all()
    result_as_dict = [dict(res) for res in result_as_dict]

    return result_as_dict


In [16]:
get_images_from_room('test')

[{'pseudo': 'Bzzz',
  'prompt': 'A supernova inside a glass bottle on a wooden table. Elegant, intricate, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by greg rutkowski',
  'height': 894.0,
  'width': 789.0,
  'posX': 262.0,
  'posY': 274.0,
  'timestamp': datetime.datetime(2022, 10, 7, 21, 14, 8),
  'status': 'generated',
  'path': 'test/20221007211419905237-A supernova inside a glass bottle on a wooden table. Elegant, intricate, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by greg rutkowski.jpeg'},
 {'pseudo': 'Bzzz',
  'prompt': 'A supernova inside a glass bottle on a wooden table. Digital painting, illustration, art by John Schoenherr',
  'height': 901.0,
  'width': 798.0,
  'posX': 1274.0,
  'posY': 269.0,
  'timestamp': datetime.datetime(2022, 10, 7, 21, 23, 2),
  'status': 'generated',
  'path': 'test/20221007212312422622-A supernova inside a glass bottle on a wooden table. Digital painting, illustration, 

In [4]:
db_operations.get_images_from_room('test')

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'localhost' ([Errno 2] No such file or directory)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [5]:
import sqlalchemy


In [6]:
?sqlalchemy.engine.url.URL.create

Signature:
sqlalchemy.engine.url.URL.create(
    drivername,
    username=None,
    password=None,
    host=None,
    port=None,
    database=None,
    query=immutabledict({}),
)
Docstring:
Create a new :class:`_engine.URL` object.

:param drivername: the name of the database backend. This name will
  correspond to a module in sqlalchemy/databases or a third party
  plug-in.
:param username: The user name.
:param password: database password.  Is typically a string, but may
  also be an object that can be stringified with ``str()``.

  .. note::  A password-producing object will be stringified only
     **once** per :class:`_engine.Engine` object.  For dynamic password
     generation per connect, see :ref:`engines_dynamic_tokens`.

:param host: The name of the host.
:param port: The port number.
:param database: The database name.
:param query: A dictionary of string keys to string values to be passed
  to the dialect and/or the DBAPI upon connect.   To specify non-string
  parameters 